<a href="https://colab.research.google.com/github/pangeab-blip/EvGeo-Exercises/blob/main/CO2_Buildup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# CO2 Input-only Atmosphere–Ocean Mini-Model (Colab/Jupyter, single cell)
# Two boxes: Atmosphere (ppm CO2) and Ocean (GtCO2)
# Only source: volcanic degassing (MtCO2/yr). No sinks.
# Air–sea exchange relaxes A toward the equilibrium partition f_atm_eq of the total carbon,
# with an exchange timescale tau_ex (years).
#
# Units and conversions:
#   1 ppm CO2 in the atmosphere ≈ 7.8 GtCO2  ≈ 7800 MtCO2
# We keep A in ppm, O in GtCO2, fluxes in MtCO2/yr.

import numpy as np
import math
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider

PPM_TO_GtCO2 = 7.8
PPM_TO_MtCO2 = 7800.0
GtCO2_TO_PPM = 1.0 / PPM_TO_GtCO2

def run_model(
    A0_ppm=420.0,                # initial atmospheric CO2 (ppm)
    O0_Gt=38000.0*44.0/12.0,     # initial ocean inventory in GtCO2 (≈ 38000 PgC → GtCO2)
    f_atm_eq=0.023,              # equilibrium atmospheric fraction of total carbon (in ppm-units)
    tau_ex=500.0,                # air–sea exchange timescale (years)
    D0_Mt=300.0,                 # degassing start value (MtCO2/yr)
    D1_Mt=300.0,                 # degassing end value for a linear ramp (MtCO2/yr)
    t_ramp=0,                    # ramp duration (years); 0 → constant D0
    years=5000,                  # total simulation time (years)
    target_ppm=560.0,            # target atmospheric CO2 (ppm)
    ):
    dt = 1.0  # 1 year time step
    n = int(years) + 1
    t = np.arange(n, dtype=float)

    A = np.zeros(n, dtype=float)
    O = np.zeros(n, dtype=float)
    A[0] = A0_ppm
    O[0] = O0_Gt

    # Prepare degassing series D(t) in MtCO2/yr (linear ramp from D0 to D1 over t_ramp, then hold)
    D = np.full(n, D0_Mt, dtype=float)
    if t_ramp > 0:
        ramp_idx = min(t_ramp, years)
        D[:ramp_idx+1] = np.linspace(D0_Mt, D1_Mt, ramp_idx+1)
        if ramp_idx+1 < n:
            D[ramp_idx+1:] = D1_Mt

    # Time stepping
    hit_index = None
    for i in range(1, n):
        # Convert degassing to ppm/yr
        D_ppm = D[i-1] / PPM_TO_MtCO2  # MtCO2/yr → ppm/yr

        # Total carbon in "ppm-units": C_tot_ppm = A + O/7.8
        C_tot_ppm = A[i-1] + O[i-1] * GtCO2_TO_PPM
        A_eq = f_atm_eq * C_tot_ppm

        # Exchange flux in ppm/yr: positive = air -> ocean
        F_ex_ppm = (A[i-1] - A_eq) / tau_ex

        # Update boxes
        A[i] = A[i-1] + (D_ppm - F_ex_ppm) * dt
        O[i] = O[i-1] + (F_ex_ppm * PPM_TO_GtCO2) * dt  # ocean gains when atmosphere loses to ocean

        # Detect first time reaching target
        if hit_index is None and A[i] >= target_ppm:
            hit_index = i

    # Diagnostics
    reached = hit_index is not None
    if reached:
        years_to_target = hit_index
    else:
        years_to_target = None

    # Print summary
    print("=== Input-only Atmosphere–Ocean CO2 Model ===")
    print(f"Initial A = {A0_ppm:.1f} ppm, Initial O = {O0_Gt:.0f} GtCO2")
    print(f"Equilibrium atmospheric fraction f_atm_eq = {f_atm_eq:.4f}, exchange timescale tau_ex = {tau_ex:.0f} yr")
    if t_ramp > 0:
        print(f"Degassing D(t): ramp {D0_Mt:.0f} → {D1_Mt:.0f} MtCO2/yr over {t_ramp} yr, then hold")
    else:
        print(f"Degassing D: constant {D0_Mt:.0f} MtCO2/yr")
    print(f"Target A = {target_ppm:.1f} ppm; simulation length = {years} yr")

    if reached:
        print(f"Reached target in ~ {years_to_target} years "
              f"(t ≈ {years_to_target/1000:.2f} kyr).")
    else:
        print("Target NOT reached within the simulated time.")

    # Plot 1: Atmospheric CO2 (ppm) vs time
    plt.figure(figsize=(7,4))
    plt.plot(t, A)
    plt.xlabel("Time (years)")
    plt.ylabel("Atmospheric CO$_2$ (ppm)")
    plt.title("Atmospheric CO$_2$ (ppm) over time")
    plt.grid(True, linestyle=":")
    if reached:
        plt.axvline(years_to_target, linestyle="--")
    plt.show()

    # Plot 2: Atmospheric fraction of total carbon vs time
    f_atm_t = A / (A + O * GtCO2_TO_PPM)
    plt.figure(figsize=(7,4))
    plt.plot(t, f_atm_t)
    plt.xlabel("Time (years)")
    plt.ylabel("Atmospheric fraction of total carbon")
    plt.title("A / (A + O/7.8) over time")
    plt.grid(True, linestyle=":")
    plt.show()

    return t, A, O, D, years_to_target

# Interactive UI
interact(
    run_model,
    A0_ppm=FloatSlider(value=420.0, min=200.0, max=800.0, step=1.0, description="A0 (ppm)"),
    O0_Gt=FloatSlider(value=38000.0*44.0/12.0, min=1.0e5, max=2.0e6, step=1.0e4, description="O0 (GtCO2)"),
    f_atm_eq=FloatSlider(value=0.023, min=0.005, max=0.10, step=0.001, description="f_atm_eq"),
    tau_ex=FloatSlider(value=500.0, min=50.0, max=3000.0, step=10.0, description="tau_ex (yr)"),
    D0_Mt=FloatSlider(value=300.0, min=0.0, max=2000.0, step=10.0, description="D0 (Mt/yr)"),
    D1_Mt=FloatSlider(value=300.0, min=0.0, max=2000.0, step=10.0, description="D1 (Mt/yr)"),
    t_ramp=IntSlider(value=0, min=0, max=10000, step=10, description="ramp (yr)"),
    years=IntSlider(value=5000, min=100, max=50000, step=100, description="years"),
    target_ppm=FloatSlider(value=560.0, min=300.0, max=2000.0, step=5.0, description="target (ppm)"),
);

interactive(children=(FloatSlider(value=420.0, description='A0 (ppm)', max=800.0, min=200.0, step=1.0), FloatS…